# Tutorial 2: Dynamic Entry-Exit Game - Arcidiacono & Miller (Ecma, 2011), Section 7.2

This notebook replicates the structural estimation from:

**"Conditional Choice Probability Estimation of Dynamic Discrete Choice Models With Unobserved Heterogeneity"**
*Arcidiacono & Miller, Econometrica (2011)*

---

## Overview

This notebook implements a complete framework for:
1. **Solving** Markov Perfect Equilibria in dynamic oligopoly entry-exit games
2. **Simulating** panel data from equilibrium play
3. **Estimating** structural parameters using CCP-based methods

Unlike single-agent problems (like Rust's bus engine), this model features:
- **Strategic interaction**: Each firm's optimal decision depends on competitors' actions
- **Equilibrium computation**: CCPs must form a Markov Perfect Equilibrium (MPE)
- **Entry costs**: Non-incumbents face sunk costs to enter the market

### Notebook Structure

| Part | Content |
|------|---------|
| 0 | Configuration & true parameters |
| 1 | Core building blocks (utility, state space, transitions) |
| 2 | Model components (flow utility, finite dependence, aggregation) |
| 3 | Equilibrium solver (policy iteration on CCPs) |
| 4 | Data simulation from equilibrium |
| 5 | Estimation methods (Two-Stage, NPL, NFXP) |
| 6 | Comparison and Monte Carlo study |
| 7 | Summary and takeaways |


-----
## Model Overview

### Economic Setting

- **Markets**: $M$ independent markets, each observed for $T$ periods
- **Players**: $I$ symmetric potential firms per market
- **Timing**: Each period, firms simultaneously decide whether to operate

### State Variables

The full state is $x_t = (x_1, x_{2t}, s_t)$ where:

| Variable | Description | Values |
|----------|-------------|--------|
| $x_1$ | Permanent market characteristic | $\{0, 1, ..., 9\}$ (10 values) |
| $x_{2t}$ | Incumbency vector $(d_{t-1}^{(1)}, ..., d_{t-1}^{(I)})$ | Each $\in \{0,1\}$ |
| $s_t$ | Transitory demand shock | $\{0, 1, ..., 4\}$ (5 values) |

**Key insight for symmetric equilibrium**: With identical firms, we aggregate $x_{2t}$ into:
- $\ell_t^{(i)} \in \{0, 1\}$: Own incumbency (was I active last period?)
- $n_t = \sum_{j \neq i} \ell_t^{(j)}$: Number of OTHER incumbent rivals

This reduces the state space from exponential in $I$ to polynomial.

### Actions

$d_t \in \{0, 1\}$:
- $d = 1$: **Operate** (enter if non-incumbent, stay if incumbent)
- $d = 0$: **Not operate** (exit if incumbent, stay out if non-incumbent)

### Flow Utility

For a firm choosing to operate ($d = 1$), matching MATLAB `io1009`:

$$U_1 = \theta_0 + \theta_1 \cdot x_1 + \theta_2 \cdot s_t + \theta_3 \cdot n_{\text{rivals}} + \theta_4 \cdot \mathbf{1}\{\text{entrant}\} + \varepsilon_1$$

where:
- $\theta_0 = 0$: Baseline profit (intercept)
- $\theta_1 = -0.05$: Market characteristic effect (**negative** per MATLAB `bx`)
- $\theta_2 = 0.25$: Demand shock effect (`bss`)
- $\theta_3 = -0.20$: Competition effect (`bnf`, negative: more rivals reduce profit)
- $\theta_4 = -1.5$: Entry cost (`be`, negative: sunk cost for entrants)
- $\varepsilon_1$: Type-I extreme value shock

Choosing not to operate ($d = 0$) yields: $U_0 = \varepsilon_0$

### Key Insight: Finite Dependence (Renewal Property)

**Exit is a terminal action**: Once a firm exits ($d=0$), its continuation value depends only on returning as a non-incumbent ($\ell'=0$). This creates a **finite dependence** structure that enables Hotz-Miller inversion without computing the full value function.


---

## Part 0: Configuration & Parameters

In [ ]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from scipy.optimize import minimize, minimize_scalar
from scipy.special import comb
from dataclasses import dataclass, field
from typing import Tuple, Dict, List, Optional, Callable
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# Try to import tqdm, use simple fallback if not available
try:
    from tqdm.auto import tqdm
except ImportError:
    def tqdm(iterable, desc=None):
        """Simple fallback for tqdm."""
        if desc:
            print(f"{desc}...")
        return iterable

# Constants
EULER_CONSTANT = 0.5772156649015329

In [ ]:
@dataclass
class ModelParameters:
    """Structural parameters for the entry-exit game.
    
    Flow utility for operating (d=1):
        U_1 = θ_0 + θ_1·x_1 + θ_2·s_t + θ_3·Σ_{i'≠i} d_t^{(i')} + θ_4·1{entrant} + ε
    
    where:
        - θ_0: baseline profitability (intercept)
        - θ_1: effect of permanent market characteristic
        - θ_2: effect of demand shock state
        - θ_3: competition effect (negative: more rivals reduce profit)
        - θ_4: entry cost (negative, paid only by non-incumbents)
    """
    theta_0: float = 0.0      # Intercept
    theta_1: float = -0.05    # Market characteristic effect (negative per MATLAB)
    theta_2: float = 0.25     # Demand shock effect
    theta_3: float = -0.20    # Competition effect
    theta_4: float = -1.5     # Entry cost
    beta: float = 0.9         # Discount factor
    pi: float = 0.7           # Demand shock persistence
    I: int = 6                # Number of potential firms
    n_x1: int = 10            # x_1 ∈ {1,...,10} per specification
    n_s: int = 5              # s_t ∈ {1,...,5} per specification
    
    def theta_vector(self) -> np.ndarray:
        """Return structural parameters as array."""
        return np.array([self.theta_0, self.theta_1, self.theta_2, 
                        self.theta_3, self.theta_4])
    
    @classmethod
    def from_vector(cls, theta: np.ndarray, **kwargs) -> 'ModelParameters':
        """Create ModelParameters from theta vector."""
        return cls(
            theta_0=theta[0], theta_1=theta[1], theta_2=theta[2],
            theta_3=theta[3], theta_4=theta[4], **kwargs
        )


@dataclass
class SimulationConfig:
    """Configuration for data simulation."""
    M: int = 200                  # Number of markets (reduced for larger state space)
    T: int = 50                   # Total periods
    burn_in: int = 20             # Burn-in periods to discard
    seed: Optional[int] = 42      # Random seed
    
    @property
    def T_observed(self) -> int:
        return self.T - self.burn_in

@dataclass 
class EstimationConfig:
    """Configuration for estimation routines."""
    tolerance: float = 1e-8
    max_iter: int = 500
    verbose: bool = True


In [ ]:
# Default parameters matching Arcidiacono & Miller (2011) Table II
DEFAULT_PARAMS = ModelParameters()
DEFAULT_SIM_CONFIG = SimulationConfig()
DEFAULT_EST_CONFIG = EstimationConfig()

# Display parameters in table format matching the paper
print("=" * 70)
print("Model Parameters (Arcidiacono & Miller, 2011)")
print("=" * 70)

structural_params = [
    ['Intercept', 'θ₀', f'{DEFAULT_PARAMS.theta_0:.2f}'],
    ['Market characteristic', 'θ₁', f'{DEFAULT_PARAMS.theta_1:.2f}'],
    ['Demand state', 'θ₂', f'{DEFAULT_PARAMS.theta_2:.2f}'],
    ['Competition effect', 'θ₃', f'{DEFAULT_PARAMS.theta_3:.2f}'],
    ['Entry cost', 'θ₄', f'{DEFAULT_PARAMS.theta_4:.2f}'],
]
print("\nStructural Parameters:")
print(tabulate(structural_params, 
               headers=['Parameter', 'Symbol', 'Value'],
               tablefmt='simple'))

model_config = [
    ['Discount factor', 'β', f'{DEFAULT_PARAMS.beta:.2f}'],
    ['Demand persistence', 'π', f'{DEFAULT_PARAMS.pi:.2f}'],
    ['Number of firms', 'I', f'{DEFAULT_PARAMS.I}'],
    ['Market char. values', 'n_x₁', f'{DEFAULT_PARAMS.n_x1} (x₁ ∈ {{1,...,{DEFAULT_PARAMS.n_x1}}})'],
    ['Demand state values', 'n_s', f'{DEFAULT_PARAMS.n_s} (s ∈ {{1,...,{DEFAULT_PARAMS.n_s}}})'],
]
print("\nModel Configuration:")
print(tabulate(model_config,
               headers=['Parameter', 'Symbol', 'Value'],
               tablefmt='simple'))

sim_config = [
    ['Markets', 'M', f'{DEFAULT_SIM_CONFIG.M}'],
    ['Total periods', 'T', f'{DEFAULT_SIM_CONFIG.T}'],
    ['Burn-in periods', '', f'{DEFAULT_SIM_CONFIG.burn_in}'],
    ['Observed periods', '', f'{DEFAULT_SIM_CONFIG.T_observed}'],
]
print("\nSimulation Configuration:")
print(tabulate(sim_config,
               headers=['Parameter', 'Symbol', 'Value'],
               tablefmt='simple'))

# Verify transition probabilities
print("\nDemand Shock Transition Probabilities:")
print(f"  P(s'=s | s) = π = {DEFAULT_PARAMS.pi:.2f}")
print(f"  P(s'≠s | s) = (1-π)/(n_s-1) = {(1-DEFAULT_PARAMS.pi)/(DEFAULT_PARAMS.n_s-1):.4f}")


---

## Part 1: Core Building Blocks

**Why separate Parts 1 and 2?**

We organize the code into two parts to mirror the conceptual structure of dynamic games:

| Part | Focus | Classes |
|------|-------|---------|
| **Part 1** | *Primitives* — objects that don't depend on equilibrium | StateSpace, TransitionKernel |
| **Part 2** | *Model components* — objects that use CCPs or compute equilibrium | FlowUtility, FiniteDependence, Aggregator |

This separation clarifies which objects are "given" (primitives) versus which require equilibrium computation.

### 1.1 Utility Functions

**What this does**: Defines the dataclasses for model parameters, simulation settings, and estimation options.

**Economic content**: The flow utility for operating ($d=1$) is:
$$
U_1 = \theta_0 + \theta_1 x_1 + \theta_2 s_t + \theta_3 \cdot (\text{\# active rivals}) + \theta_4 \cdot \mathbf{1}\{\text{entrant}\} + \varepsilon_1
$$

- The competition term $\theta_3 \cdot (\text{\# active rivals})$ depends on rivals' *current* choices, which in equilibrium are determined by CCPs
- This is why we need to solve for CCPs before evaluating expected utilities


In [ ]:
def logit_prob(v: np.ndarray) -> np.ndarray:
    """
    Compute logit probability: exp(v) / (1 + exp(v)).
    Numerically stable implementation.
    """
    v = np.asarray(v)
    result = np.zeros_like(v, dtype=float)
    pos = v >= 0
    result[pos] = 1.0 / (1.0 + np.exp(-v[pos]))
    result[~pos] = np.exp(v[~pos]) / (1.0 + np.exp(v[~pos]))
    return result

def log_odds(p: np.ndarray) -> np.ndarray:
    """
    Compute log-odds: log(p / (1-p)).
    """
    p = np.clip(p, 1e-10, 1 - 1e-10)
    return np.log(p / (1 - p))

def emax_logit(p: np.ndarray) -> np.ndarray:
    """
    Compute E[max(eps_0, v + eps_1)] for Type I EV shocks.
    
    Using Hotz-Miller: E[max] = gamma - log(1-p) where p = P(d=1).
    This is the ex-ante value function V(x) when v_0 is normalized to 0.
    """
    p = np.clip(p, 1e-10, 1 - 1e-10)
    return EULER_CONSTANT - np.log(1 - p)


def logsum(v0: np.ndarray, v1: np.ndarray) -> np.ndarray:
    """
    Compute log(exp(v0) + exp(v1)) in a numerically stable way.
    """
    max_v = np.maximum(v0, v1)
    return max_v + np.log(np.exp(v0 - max_v) + np.exp(v1 - max_v))


# Test utilities
print("Utility function tests:")
print(f"  logit_prob(0) = {logit_prob(0):.4f} (should be 0.5)")
print(f"  logit_prob(2) = {logit_prob(2):.4f}")
print(f"  log_odds(0.5) = {log_odds(0.5):.4f} (should be 0)")
print(f"  emax_logit(0.5) = {emax_logit(0.5):.4f}")

### 1.2 StateSpace Class

**What this does**: Enumerates all possible states from a single firm's perspective.

**Key insight for symmetric equilibrium**: With $I$ symmetric firms, we don't track each firm's identity. Instead, we track:
- $x_1 \in \{0, ..., n_{x_1}-1\}$: Permanent market characteristic
- $\ell \in \{0, 1\}$: Own incumbency status (was I active last period?)
- $n_{\text{rivals}} \in \{0, ..., I-1\}$: Number of *other* firms who were incumbents
- $s \in \{0, ..., n_s-1\}$: Demand shock state

**State space size**: $n_{x_1} \times 2 \times I \times n_s$ states per firm.

With default parameters: $10 \times 2 \times 6 \times 5 = 600$ states.


In [ ]:
class StateSpace:
    """Manages state space for symmetric equilibrium.
    
    States are tuples: (x1, own_incumbent, n_rivals, s)
    - x1: permanent market characteristic index
    - own_incumbent: 1 if firm was active last period, 0 otherwise
    - n_rivals: number of other firms who were incumbents
    - s: demand shock state index
    
    Indexing: We use a flat index for efficient array operations.
    The ordering is: iterate over s (fastest), then n_rivals, then own_inc, then x1 (slowest).
    """
    
    def __init__(self, params: ModelParameters):
        self.params = params
        self.n_x1 = params.n_x1
        self.n_s = params.n_s
        self.I = params.I
        self.max_rivals = params.I - 1  # Max number of other incumbents
        
        # Enumerate all states
        self._states = []
        self._index_map = {}
        
        idx = 0
        for x1 in range(self.n_x1):
            for own_inc in [0, 1]:
                for n_rivals in range(self.I):  # 0 to I-1 other incumbents
                    for s in range(self.n_s):
                        state = (x1, own_inc, n_rivals, s)
                        self._states.append(state)
                        self._index_map[state] = idx
                        idx += 1
        
        self._n_states = len(self._states)
    
    def enumerate_states(self) -> List[Tuple[int, int, int, int]]:
        """Return list of all states."""
        return self._states.copy()
    
    def state_to_index(self, state: Tuple[int, int, int, int]) -> int:
        """Convert state tuple to flat index."""
        return self._index_map[state]
    
    def index_to_state(self, idx: int) -> Tuple[int, int, int, int]:
        """Convert flat index to state tuple."""
        return self._states[idx]
    
    @property
    def n_states(self) -> int:
        """Total number of states."""
        return self._n_states
    
    def get_state_arrays(self) -> Dict[str, np.ndarray]:
        """Return arrays of state components for vectorized operations."""
        states = np.array(self._states)
        return {
            'x1': states[:, 0],
            'own_inc': states[:, 1],
            'n_rivals': states[:, 2],
            's': states[:, 3]
        }
    
    def __repr__(self) -> str:
        return f"StateSpace(n_states={self.n_states}, I={self.I}, n_x1={self.n_x1}, n_s={self.n_s})"


# Test StateSpace
ss = StateSpace(DEFAULT_PARAMS)
print(f"StateSpace: {ss}")
print(f"\nSample states:")
for i in [0, 1, ss.n_states // 2, ss.n_states - 1]:
    state = ss.index_to_state(i)
    print(f"  Index {i}: (x1={state[0]}, own_inc={state[1]}, n_rivals={state[2]}, s={state[3]})")

### 1.3 TransitionKernel Class

**What this does**: Defines the Markov transition probabilities for the exogenous demand shock $s_t$.

**From the specification**: The demand shock follows a first-order Markov chain with persistence $\pi$:
$$P(s_{t+1} | s_t) = \begin{cases} \pi & \text{if } s_{t+1} = s_t \\ (1-\pi)/(n_s - 1) & \text{otherwise} \end{cases}$$

With $\pi = 0.7$ and $n_s = 5$:
- Stay in same state: 70%
- Transition to each other state: 7.5%

This captures the idea that demand conditions are persistent but stochastic.


In [ ]:
class TransitionKernel:
    """Manages demand shock transition probabilities.
    
    The demand shock s follows a Markov chain with persistence parameter pi.
    P(s' = s | s) = pi
    P(s' != s | s) = (1 - pi) / (n_s - 1) for each other state
    """
    
    def __init__(self, n_s: int, pi: float):
        self.n_s = n_s
        self.pi = pi
        self._build_transition_matrix()
    
    def _build_transition_matrix(self):
        """Construct the transition matrix."""
        self._trans_mat = np.full((self.n_s, self.n_s), 
                                   (1 - self.pi) / (self.n_s - 1))
        np.fill_diagonal(self._trans_mat, self.pi)
    
    def transition_matrix(self) -> np.ndarray:
        """Return the full transition matrix P[s, s']."""
        return self._trans_mat.copy()
    
    def prob(self, s_next: int, s_current: int) -> float:
        """P(s' = s_next | s = s_current)."""
        return self._trans_mat[s_current, s_next]
    
    def __repr__(self) -> str:
        return f"TransitionKernel(n_s={self.n_s}, pi={self.pi})"


# Test TransitionKernel
tk = TransitionKernel(DEFAULT_PARAMS.n_s, DEFAULT_PARAMS.pi)
print(f"{tk}")
print(f"\nTransition matrix:")
print(tk.transition_matrix())
print(f"\nRow sums (should be 1): {tk.transition_matrix().sum(axis=1)}")

---

## Part 2: Model Components

Now we build objects that compute model quantities from the primitives.

**Key challenge**: The flow utility depends on rivals' actions, but rivals' actions depend on CCPs, and CCPs depend on utilities. This creates a fixed-point problem we solve in Part 3.

### 2.1 FlowUtility Class

**What this does**: Computes flow payoffs for operating ($d=1$) given:
1. State variables $(x_1, s, \ell)$
2. Number of active rivals (or CCPs to compute expected number)

**Two modes**:
- `primitive(x1, s, n_active_rivals, is_entrant)`: Flow utility given realized rival actions
- `compute_expected_flow_utility(ccps)`: Expected utility integrating over rival choices using CCPs

**Computing expected rivals**: Given $n_{\text{inc}}$ rival incumbents and $n_{\text{ent}}$ rival potential entrants:
$$\mathbb{E}[\text{\# active rivals}] = n_{\text{inc}} \cdot p_{\text{stay}} + n_{\text{ent}} \cdot p_{\text{enter}}$$

where $p_{\text{stay}}$ and $p_{\text{enter}}$ come from the CCPs.


In [ ]:
class FlowUtility:
    """Computes flow utilities for the entry-exit game.
    
    Two types of utilities:
    1. primitive(state, is_entrant): U_1 given realized rival actions
    2. expected(state, rival_ccps): E[U_1] integrating over rival choices
    
    Convention:
    - d=0: exit/stay out (normalized to 0 utility)
    - d=1: operate (enter if non-incumbent, stay if incumbent)
    """
    
    def __init__(self, params: ModelParameters):
        self.params = params
        self.theta = params.theta_vector()
    
    def primitive(self, x1: int, s: int, n_active_rivals: int, 
                  is_entrant: bool) -> float:
        """Flow utility for operating given realized number of active rivals.
        
        U_1 = theta_0 + theta_1 * x1 + theta_2 * s + theta_3 * n_active_rivals 
              + theta_4 * is_entrant
        """
        u = (self.params.theta_0 + 
             self.params.theta_1 * x1 + 
             self.params.theta_2 * s + 
             self.params.theta_3 * n_active_rivals +
             self.params.theta_4 * int(is_entrant))
        return u
    
    def primitive_vectorized(self, state_arrays: Dict[str, np.ndarray]) -> np.ndarray:
        """Vectorized flow utility computation.
        
        Returns U_1 for each state, treating n_rivals as the expected number
        of active rivals (used in equilibrium computation).
        """
        x1 = state_arrays['x1']
        s = state_arrays['s']
        own_inc = state_arrays['own_inc']
        n_rivals = state_arrays['n_rivals']  # Number of rival incumbents
        is_entrant = 1 - own_inc
        
        # Note: n_rivals here is the number of OTHER incumbents.
        # The expected number of active rivals depends on their CCPs.
        # For now, we use n_rivals as a placeholder.
        u = (self.params.theta_0 + 
             self.params.theta_1 * x1 + 
             self.params.theta_2 * s + 
             self.params.theta_3 * n_rivals +
             self.params.theta_4 * is_entrant)
        return u
    
    def expected_n_active_rivals(self, n_rival_inc: int, n_rival_ent: int,
                                  p_stay: float, p_enter: float) -> float:
        """Expected number of active rivals given CCPs.
        
        n_rival_inc: number of rival incumbents
        n_rival_ent: number of rival potential entrants
        p_stay: probability incumbent stays
        p_enter: probability entrant enters
        """
        return n_rival_inc * p_stay + n_rival_ent * p_enter


# Test FlowUtility
fu = FlowUtility(DEFAULT_PARAMS)
print("Flow utility tests:")
print(f"  U(x1=2, s=1, n_rivals=3, entrant=False) = {fu.primitive(2, 1, 3, False):.4f}")
print(f"  U(x1=2, s=1, n_rivals=3, entrant=True) = {fu.primitive(2, 1, 3, True):.4f}")
print(f"  Difference (entry cost) = {fu.primitive(2, 1, 3, True) - fu.primitive(2, 1, 3, False):.4f}")

### 2.2 FiniteDependence Class

**What this does**: Implements the Hotz-Miller inversion exploiting the *finite dependence* (renewal) property.

**Key insight**: Exit ($d=0$) is a *terminal* action. If a firm exits, its continuation value only depends on returning as a non-incumbent ($\ell'=0$), regardless of current incumbency status.

This means we can compute the continuation value difference *without* solving the full Bellman equation:
$$\bar{V}(\ell=1, \cdot) - \bar{V}(\ell=0, \cdot) = \gamma - \ln(1 - p_1(\ell=1, \cdot)) - [\gamma - \ln(1 - p_1(\ell=0, \cdot))]$$

Simplifying:
$$= \ln\left(\frac{1 - p_1(\ell=0, \cdot)}{1 - p_1(\ell=1, \cdot)}\right)$$

where $\gamma \approx 0.5772$ is Euler's constant and $p_1$ is the CCP for operating.


In [ ]:
class FiniteDependence:
    """Exploits the terminal choice structure for CCP-based estimation.
    
    Key insight: When d=0 (exit/stay out) is a terminal choice with v_0 = 0,
    we can express the ex-ante value function using only CCPs:
    
        V(x) = E[max(0, v_1(x) + eps_1 - eps_0)]
             = gamma - log(1 - p_1(x))
    
    where p_1(x) = P(d=1 | x) is the CCP of operating.
    
    This eliminates the need to solve the full Bellman equation!
    """
    
    def __init__(self):
        pass
    
    def ex_ante_value(self, ccps: np.ndarray) -> np.ndarray:
        """Compute V(x) = gamma - log(1 - p(x)) from CCPs.
        
        Args:
            ccps: Array of P(d=1 | x) for each state x
            
        Returns:
            Array of ex-ante values V(x)
        """
        return emax_logit(ccps)
    
    def choice_specific_value_diff(self, ccps: np.ndarray) -> np.ndarray:
        """Compute v_1(x) - v_0(x) = log(p_1 / (1 - p_1)).
        
        From Hotz-Miller inversion.
        """
        return log_odds(ccps)


# Test FiniteDependence
fd = FiniteDependence()
test_ccps = np.array([0.3, 0.5, 0.7, 0.9])
print("Finite dependence tests:")
print(f"  CCPs: {test_ccps}")
print(f"  V(x) = gamma - log(1-p): {fd.ex_ante_value(test_ccps)}")
print(f"  v_1 - v_0 = log(p/(1-p)): {fd.choice_specific_value_diff(test_ccps)}")

### 2.3 BinomialAggregator Class

**What this does**: Computes transition probabilities for the *aggregate* state (number of incumbents) from individual CCPs.

**Why needed**: With symmetric firms, we don't track individual identities. Given:
- $n$ current incumbents with CCPs $p_{\text{stay}}$ (prob. incumbent operates)
- $I - n$ current entrants with CCPs $p_{\text{enter}}$ (prob. entrant enters)

The number of firms active next period follows a convolution of two binomials:
$$P(n' | n, p_{\text{stay}}, p_{\text{enter}}) = \sum_{k=0}^{\min(n', n)} P(k \text{ stay}) \cdot P(n'-k \text{ enter})$$

where:
- $P(k \text{ stay}) = \binom{n}{k} p_{\text{stay}}^k (1-p_{\text{stay}})^{n-k}$
- $P(n'-k \text{ enter}) = \binom{I-n}{n'-k} p_{\text{enter}}^{n'-k} (1-p_{\text{enter}})^{I-n-(n'-k)}$


In [ ]:
class BinomialAggregator:
    """Aggregates individual choices to distribution over number of active firms.
    
    Given individual CCPs (p_enter for entrants, p_stay for incumbents),
    computes the distribution over the total number of active firms next period.
    """
    
    def __init__(self, I: int):
        self.I = I
        self._precompute_binomial_coefficients()
    
    def _precompute_binomial_coefficients(self):
        """Precompute binomial coefficients C(n, k) for n, k <= I."""
        self._binom = np.zeros((self.I + 1, self.I + 1))
        for n in range(self.I + 1):
            for k in range(n + 1):
                self._binom[n, k] = comb(n, k, exact=True)
    
    def compute_n_active_distribution(self, n_incumbents: int, n_entrants: int,
                                       p_stay: float, p_enter: float) -> np.ndarray:
        """Compute P(N' = k) for k = 0, ..., I.
        
        Args:
            n_incumbents: Number of incumbents (among rivals)
            n_entrants: Number of potential entrants (among rivals)
            p_stay: Probability each incumbent stays
            p_enter: Probability each entrant enters
            
        Returns:
            Array of probabilities P(N' = k) for k = 0, ..., I
        """
        # Distribution over # of incumbents who stay
        P_inc = np.zeros(n_incumbents + 1)
        for k in range(n_incumbents + 1):
            P_inc[k] = (self._binom[n_incumbents, k] * 
                       (p_stay ** k) * ((1 - p_stay) ** (n_incumbents - k)))
        
        # Distribution over # of entrants who enter
        P_ent = np.zeros(n_entrants + 1)
        for j in range(n_entrants + 1):
            P_ent[j] = (self._binom[n_entrants, j] * 
                       (p_enter ** j) * ((1 - p_enter) ** (n_entrants - j)))
        
        # Convolve to get total number of active rivals
        P_total = np.zeros(self.I + 1)
        for k in range(n_incumbents + 1):
            for j in range(n_entrants + 1):
                P_total[k + j] += P_inc[k] * P_ent[j]
        
        return P_total


# Test BinomialAggregator
ba = BinomialAggregator(DEFAULT_PARAMS.I)
print("Binomial aggregation test:")
print(f"  3 incumbents (p_stay=0.8), 2 entrants (p_enter=0.3)")
dist = ba.compute_n_active_distribution(3, 2, 0.8, 0.3)
for k, p in enumerate(dist):
    if p > 0.001:
        print(f"  P(N' = {k}) = {p:.4f}")
print(f"  Sum = {dist.sum():.6f}")

### 2.4 EntryExitGame Class

**What this does**: The main model class that combines all components and implements the CCP operator $\Psi(p)$.

**The CCP operator**: Given current CCPs $p$, the operator $\Psi$ computes updated CCPs:

1. **Compute expected flow utilities** $u_1(x)$ integrating over rival choices
2. **Compute continuation values** using finite dependence
3. **Compute choice-specific values**: $v_1(x) - v_0(x) = u_1(x) + \beta \cdot [\text{CV diff}]$
4. **Apply logit formula**: $\Psi(p)(x) = \frac{\exp(v_1(x))}{\exp(v_0(x)) + \exp(v_1(x))} = \frac{1}{1 + \exp(-(v_1 - v_0))}$

**Equilibrium**: CCPs form an MPE if $p^* = \Psi(p^*)$ (fixed point).


In [ ]:
class EntryExitGame:
    """Main model class for the dynamic entry-exit game.
    
    Combines state space, utilities, transitions, and finite dependence
    to define the full game structure.
    """
    
    def __init__(self, params: ModelParameters):
        self.params = params
        self.state_space = StateSpace(params)
        self.flow_utility = FlowUtility(params)
        self.transition_kernel = TransitionKernel(params.n_s, params.pi)
        self.finite_dep = FiniteDependence()
        self.binom_agg = BinomialAggregator(params.I)
        
        # Cache state arrays for vectorized operations
        self._state_arrays = self.state_space.get_state_arrays()
    
    def compute_expected_flow_utility(self, ccps: np.ndarray) -> np.ndarray:
        """Compute expected flow utility u_1(x) integrating over rival choices.
        
        For each state x = (x1, own_inc, n_rivals, s):
        - n_rivals is the number of OTHER incumbents
        - Expected # active rivals = n_rivals * p_stay + (I-1-n_rivals) * p_enter
        
        Args:
            ccps: Array of CCPs indexed by state
            
        Returns:
            Array of expected flow utilities for d=1
        """
        n_states = self.state_space.n_states
        u_expected = np.zeros(n_states)
        
        for idx in range(n_states):
            x1, own_inc, n_rival_inc, s = self.state_space.index_to_state(idx)
            n_rival_ent = self.params.I - 1 - n_rival_inc
            is_entrant = (own_inc == 0)
            
            # Get average CCPs for rivals (simplified: use same-state CCPs)
            # This is an approximation - in full model, would integrate over rival states
            # For incumbents: use CCP at (x1, own_inc=1, *, s)
            # For entrants: use CCP at (x1, own_inc=0, *, s)
            
            # Simple approximation: average over n_rivals dimension
            p_stay_avg = 0.0
            p_enter_avg = 0.0
            count_inc = 0
            count_ent = 0
            
            for n_r in range(self.params.I):
                idx_inc = self.state_space.state_to_index((x1, 1, n_r, s))
                idx_ent = self.state_space.state_to_index((x1, 0, n_r, s))
                p_stay_avg += ccps[idx_inc]
                p_enter_avg += ccps[idx_ent]
                count_inc += 1
                count_ent += 1
            
            p_stay_avg /= count_inc
            p_enter_avg /= count_ent
            
            # Expected number of active rivals
            exp_n_active = n_rival_inc * p_stay_avg + n_rival_ent * p_enter_avg
            
            # Flow utility
            u_expected[idx] = self.flow_utility.primitive(x1, s, exp_n_active, is_entrant)
        
        return u_expected
    
    def compute_continuation_value(self, ccps: np.ndarray) -> np.ndarray:
        """Compute continuation value E[V(x') | x, d=1] - E[V(x') | x, d=0].
        
        Using finite dependence: V(x') = gamma - log(1 - p_1(x'))
        
        The difference in continuation values comes from:
        - If operate (d=1): next period own_inc=1, face some n_rivals distribution
        - If exit (d=0): next period own_inc=0, face some n_rivals distribution
        """
        n_states = self.state_space.n_states
        cont_val_diff = np.zeros(n_states)
        trans_s = self.transition_kernel.transition_matrix()
        
        # V(x') for all states
        V_next = self.finite_dep.ex_ante_value(ccps)
        
        for idx in range(n_states):
            x1, own_inc, n_rival_inc, s = self.state_space.index_to_state(idx)
            n_rival_ent = self.params.I - 1 - n_rival_inc
            
            # Get average CCPs for rivals
            p_stay_avg = 0.0
            p_enter_avg = 0.0
            for n_r in range(self.params.I):
                idx_inc = self.state_space.state_to_index((x1, 1, n_r, s))
                idx_ent = self.state_space.state_to_index((x1, 0, n_r, s))
                p_stay_avg += ccps[idx_inc]
                p_enter_avg += ccps[idx_ent]
            p_stay_avg /= self.params.I
            p_enter_avg /= self.params.I
            
            # Distribution over # of rival incumbents next period
            P_n_rival_next = self.binom_agg.compute_n_active_distribution(
                n_rival_inc, n_rival_ent, p_stay_avg, p_enter_avg
            )
            
            # E[V(x') | d=1] - E[V(x') | d=0]
            EV_if_operate = 0.0
            EV_if_exit = 0.0
            
            for s_next in range(self.params.n_s):
                for n_r_next in range(self.params.I):
                    # If operate: own_inc' = 1
                    idx_operate = self.state_space.state_to_index((x1, 1, n_r_next, s_next))
                    # If exit: own_inc' = 0
                    idx_exit = self.state_space.state_to_index((x1, 0, n_r_next, s_next))
                    
                    prob = trans_s[s, s_next] * P_n_rival_next[n_r_next]
                    EV_if_operate += prob * V_next[idx_operate]
                    EV_if_exit += prob * V_next[idx_exit]
            
            cont_val_diff[idx] = EV_if_operate - EV_if_exit
        
        return cont_val_diff
    
    def ccp_operator(self, ccps: np.ndarray) -> np.ndarray:
        """Apply the CCP mapping Psi(p).
        
        Given current CCPs, compute:
        1. Expected flow utility u_1(x)
        2. Continuation value difference
        3. Choice-specific value v_1(x) = u_1(x) + beta * CV_diff
        4. New CCPs p_1(x) = exp(v_1) / (1 + exp(v_1))
        """
        # Expected flow utility
        u1 = self.compute_expected_flow_utility(ccps)
        
        # Continuation value difference
        cv_diff = self.compute_continuation_value(ccps)
        
        # Choice-specific value for d=1 (relative to d=0)
        v1 = u1 + self.params.beta * cv_diff
        
        # New CCPs
        new_ccps = logit_prob(v1)
        
        return new_ccps
    
    def __repr__(self) -> str:
        return f"EntryExitGame(I={self.params.I}, states={self.state_space.n_states})"


# Test EntryExitGame
game = EntryExitGame(DEFAULT_PARAMS)
print(f"Created: {game}")

---

## Part 3: Equilibrium Solver

### How Equilibrium is Computed

**Problem**: Find CCPs $p^*$ such that $p^* = \Psi(p^*)$ where $\Psi$ is the CCP operator.

**Algorithm**: Policy function iteration (also called CCP iteration):

```
1. Initialize: p⁰ = 0.5 (or some starting guess)
2. For n = 0, 1, 2, ...
   a. Compute expected utilities given pⁿ
   b. Compute continuation values using Hotz-Miller
   c. Update CCPs: pⁿ⁺¹ = Ψ(pⁿ) via logit formula
   d. If ||pⁿ⁺¹ - pⁿ|| < tolerance: STOP
3. Return p* = pⁿ⁺¹
```

**Why it converges**: Under regularity conditions (Aguirregabiria & Mira, 2007), $\Psi$ is a contraction mapping, so iteration converges to the unique fixed point.

**Computation per iteration**:
1. For each state, integrate over rivals' choices using current CCPs → $O(n_{states} \times I)$
2. Compute finite dependence terms → $O(n_{states})$
3. Apply logit formula → $O(n_{states})$

With 600 states and 6 firms, each iteration is fast. Convergence typically takes 20-100 iterations.


In [ ]:
@dataclass
class EquilibriumResult:
    """Results from equilibrium computation."""
    ccps: np.ndarray
    n_iterations: int
    convergence_metric: float
    computation_time: float
    converged: bool
    history: Optional[List[np.ndarray]] = None


class EquilibriumSolver:
    """Solves for Markov Perfect Equilibrium via policy function iteration.
    
    Iterates p^{n+1} = Psi(p^n) until convergence.
    """
    
    def __init__(self, game: EntryExitGame, config: EstimationConfig = None):
        self.game = game
        self.config = config or EstimationConfig()
    
    def solve(self, initial_ccps: np.ndarray = None, 
              track_history: bool = False) -> EquilibriumResult:
        """Solve for equilibrium CCPs.
        
        Args:
            initial_ccps: Starting CCPs (default: 0.5 everywhere)
            track_history: If True, store CCP path for diagnostics
            
        Returns:
            EquilibriumResult with equilibrium CCPs and diagnostics
        """
        start_time = time.time()
        n_states = self.game.state_space.n_states
        
        # Initialize CCPs
        if initial_ccps is None:
            ccps = 0.5 * np.ones(n_states)
        else:
            ccps = initial_ccps.copy()
        
        history = [ccps.copy()] if track_history else None
        
        # Iterate
        for iteration in range(self.config.max_iter):
            ccps_old = ccps.copy()
            ccps = self.game.ccp_operator(ccps)
            
            # Convergence check
            diff = np.max(np.abs(ccps - ccps_old))
            
            if track_history:
                history.append(ccps.copy())
            
            if self.config.verbose and (iteration + 1) % 50 == 0:
                print(f"  Iteration {iteration + 1}: max|p - p_old| = {diff:.2e}")
            
            if diff < self.config.tolerance:
                if self.config.verbose:
                    print(f"  Converged in {iteration + 1} iterations")
                break
        else:
            if self.config.verbose:
                print(f"  Warning: Did not converge after {self.config.max_iter} iterations")
        
        computation_time = time.time() - start_time
        
        return EquilibriumResult(
            ccps=ccps,
            n_iterations=iteration + 1,
            convergence_metric=diff,
            computation_time=computation_time,
            converged=(diff < self.config.tolerance),
            history=history
        )

### Visualizing Equilibrium CCPs

The following figure shows how equilibrium CCPs vary with state variables. **What to look for:**

- **(a) Competition effect**: CCPs should *decrease* with more rivals ($\theta_3 < 0$)
- **(b) Demand effect**: CCPs should *increase* with demand shock ($\theta_2 > 0$)  
- **(c) Incumbency advantage**: Incumbents have higher CCPs than entrants due to entry cost ($\theta_4 < 0$)

The gap between incumbent and entrant CCPs reflects the sunk cost of entry.


In [ ]:
# Solve for equilibrium
print("=" * 70)
print("Solving for Markov Perfect Equilibrium...")
print("=" * 70)

solver = EquilibriumSolver(game, EstimationConfig(verbose=True, tolerance=1e-8))
eq_result = solver.solve(track_history=True)

print(f"\nEquilibrium found:")
print(f"  Iterations: {eq_result.n_iterations}")
print(f"  Convergence: {eq_result.convergence_metric:.2e}")
print(f"  Time: {eq_result.computation_time:.2f} seconds")

In [ ]:
# Visualize equilibrium CCPs
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ss = game.state_space
ccps = eq_result.ccps

# 1. CCPs by own_incumbent status and n_rivals (fixing x1=mid, s=mid)
x1_fixed = game.params.n_x1 // 2  # Middle value
s_fixed = game.params.n_s // 2    # Middle value
n_rivals_range = range(game.params.I)

ccps_incumbent = [ccps[ss.state_to_index((x1_fixed, 1, nr, s_fixed))] for nr in n_rivals_range]
ccps_entrant = [ccps[ss.state_to_index((x1_fixed, 0, nr, s_fixed))] for nr in n_rivals_range]

axes[0].plot(n_rivals_range, ccps_incumbent, 'b-o', label='Incumbent (ℓ=1)', linewidth=2, markersize=8)
axes[0].plot(n_rivals_range, ccps_entrant, 'r-s', label='Entrant (ℓ=0)', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Rival Incumbents')
axes[0].set_ylabel('P(Operate)')
axes[0].set_title(f'(a) CCPs vs Competition\n(x₁={x1_fixed}, s={s_fixed})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 1)

# 2. CCPs by demand shock s (fixing x1, n_rivals)
n_rivals_fixed = 2
s_range = range(game.params.n_s)

ccps_by_s_inc = [ccps[ss.state_to_index((x1_fixed, 1, n_rivals_fixed, s))] for s in s_range]
ccps_by_s_ent = [ccps[ss.state_to_index((x1_fixed, 0, n_rivals_fixed, s))] for s in s_range]

axes[1].plot(s_range, ccps_by_s_inc, 'b-o', label='Incumbent (ℓ=1)', linewidth=2, markersize=8)
axes[1].plot(s_range, ccps_by_s_ent, 'r-s', label='Entrant (ℓ=0)', linewidth=2, markersize=8)
axes[1].set_xlabel('Demand Shock State (s)')
axes[1].set_ylabel('P(Operate)')
axes[1].set_title(f'(b) CCPs vs Demand\n(x₁={x1_fixed}, n_rivals={n_rivals_fixed})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1)

# 3. Gap between incumbent and entrant CCPs
ccp_gap = np.array(ccps_incumbent) - np.array(ccps_entrant)
axes[2].bar(n_rivals_range, ccp_gap, color='purple', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Number of Rival Incumbents')
axes[2].set_ylabel('CCP Gap (Incumbent - Entrant)')
axes[2].set_title('(c) Incumbency Advantage\n(P(stay) - P(enter))')
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print interpretation
print("\nFigure Interpretation:")
print("-" * 60)
print("(a) CCPs decrease with more rivals (θ₃ < 0): competition reduces operating probability")
print("(b) CCPs increase with demand (θ₂ > 0): high demand attracts more firms")
print("(c) Incumbency advantage (~0.4-0.5): entry cost θ₄=-1.5 creates barrier to entry")


---

## Part 4: Data Simulation

### Visualizing Simulated Data

The following figure shows patterns in the simulated data. **What to look for:**

- **(a) Market structure**: Distribution of active firms—not too concentrated (monopoly) or too competitive
- **(b) Entry/stay rates**: Both should decrease with competition, but stay rate > entry rate (incumbency advantage)
- **(c) Demand response**: Higher demand states should have higher operate rates

These patterns are generated by firms playing equilibrium strategies.


In [ ]:
class DataSimulator:
    """Generates panel data from equilibrium CCPs."""
    
    def __init__(self, game: EntryExitGame, ccps: np.ndarray,
                 config: SimulationConfig):
        self.game = game
        self.ccps = ccps
        self.config = config
        self.trans_s = game.transition_kernel.transition_matrix()
        self.trans_s_cumsum = np.cumsum(self.trans_s, axis=1)
    
    def simulate(self) -> pd.DataFrame:
        """Simulate panel data.
        
        Returns DataFrame with columns:
        - market_id, period, firm_id
        - x1, s, own_incumbent, n_rivals
        - choice (1=operate, 0=exit/stay out)
        """
        if self.config.seed is not None:
            np.random.seed(self.config.seed)
        
        M = self.config.M
        T_total = self.config.T
        T_burn = self.config.burn_in
        I = self.game.params.I
        n_x1 = self.game.params.n_x1
        n_s = self.game.params.n_s
        ss = self.game.state_space
        
        # Storage
        records = []
        
        # For each market
        for m in range(M):
            # Draw permanent market characteristic
            x1 = np.random.randint(0, n_x1)
            
            # Initialize: demand shock and incumbency status
            s = np.random.randint(0, n_s)
            incumbent = np.zeros(I, dtype=int)  # No one is incumbent initially
            
            # Random draws for simulation
            draw_choice = np.random.rand(T_total, I)
            draw_s = np.random.rand(T_total)
            
            for t in range(T_total):
                # Count incumbents
                n_incumbents = incumbent.sum()
                
                # Each firm makes a choice
                choices = np.zeros(I, dtype=int)
                
                for i in range(I):
                    own_inc = incumbent[i]
                    # Number of OTHER incumbents
                    n_rivals = n_incumbents - own_inc
                    
                    # Get CCP
                    state_idx = ss.state_to_index((x1, own_inc, n_rivals, s))
                    p_operate = self.ccps[state_idx]
                    
                    # Make choice
                    choices[i] = 1 if draw_choice[t, i] < p_operate else 0
                    
                    # Record if past burn-in
                    if t >= T_burn:
                        records.append({
                            'market_id': m,
                            'period': t - T_burn,
                            'firm_id': i,
                            'x1': x1,
                            's': s,
                            'own_incumbent': own_inc,
                            'n_rivals': n_rivals,
                            'choice': choices[i]
                        })
                
                # Update incumbency status
                incumbent = choices.copy()
                
                # Transition demand shock
                s = np.searchsorted(self.trans_s_cumsum[s], draw_s[t])
                s = min(s, n_s - 1)
        
        return pd.DataFrame(records)
    
    def summary_statistics(self, data: pd.DataFrame) -> Dict:
        """Compute summary statistics from simulated data."""
        # Overall
        n_obs = len(data)
        operate_rate = data['choice'].mean()
        
        # By incumbency status
        entry_mask = data['own_incumbent'] == 0
        stay_mask = data['own_incumbent'] == 1
        entry_rate = data.loc[entry_mask, 'choice'].mean() if entry_mask.sum() > 0 else np.nan
        stay_rate = data.loc[stay_mask, 'choice'].mean() if stay_mask.sum() > 0 else np.nan
        exit_rate = 1 - stay_rate if not np.isnan(stay_rate) else np.nan
        
        # By x1
        operate_by_x1 = data.groupby('x1')['choice'].mean().to_dict()
        
        # By s
        operate_by_s = data.groupby('s')['choice'].mean().to_dict()
        
        # Average number of firms
        firms_per_market_period = data.groupby(['market_id', 'period'])['choice'].sum()
        avg_firms = firms_per_market_period.mean()
        
        return {
            'n_observations': n_obs,
            'operate_rate': operate_rate,
            'entry_rate': entry_rate,
            'exit_rate': exit_rate,
            'avg_firms': avg_firms,
            'operate_by_x1': operate_by_x1,
            'operate_by_s': operate_by_s
        }

In [ ]:
import time

# Simulate data
print("=" * 70)
print("Simulating data from equilibrium...")
print("=" * 70)

sim_config = SimulationConfig(M=200, T=50, burn_in=20, seed=42)
simulator = DataSimulator(game, eq_result.ccps, sim_config)

start_time = time.time()
data = simulator.simulate()
sim_time = time.time() - start_time

print(f"\nSimulation completed in {sim_time:.2f} seconds")
print(f"Generated {len(data):,} observations")
print(f"  Markets: {data['market_id'].nunique()}")
print(f"  Periods: {data['period'].nunique()}")
print(f"  Firms per market: {game.params.I}")

# Summary statistics
stats = simulator.summary_statistics(data)
print(f"\nSummary Statistics:")
print(f"  Overall operate rate: {stats['operate_rate']:.3f}")
print(f"  Entry rate (non-incumbent choosing d=1): {stats['entry_rate']:.3f}")
print(f"  Exit rate (incumbent choosing d=0): {stats['exit_rate']:.3f}")
print(f"  Average firms per market-period: {stats['avg_firms']:.2f}")

In [ ]:
# Visualize simulated data patterns
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Distribution of active firms per market-period
firms_per_mp = data.groupby(['market_id', 'period'])['choice'].sum()
axes[0].hist(firms_per_mp, bins=range(game.params.I + 2), edgecolor='black', 
             alpha=0.7, density=True, align='left', color='steelblue')
axes[0].set_xlabel('Number of Active Firms')
axes[0].set_ylabel('Density')
axes[0].set_title('(a) Active Firms per Market-Period')
axes[0].set_xticks(range(game.params.I + 1))

# 2. Entry/exit rates by n_rivals
entry_by_nr = data[data['own_incumbent'] == 0].groupby('n_rivals')['choice'].mean()
stay_by_nr = data[data['own_incumbent'] == 1].groupby('n_rivals')['choice'].mean()

axes[1].plot(entry_by_nr.index, entry_by_nr.values, 'g-o', label='Entry rate (ℓ=0→d=1)', linewidth=2, markersize=8)
axes[1].plot(stay_by_nr.index, stay_by_nr.values, 'b-s', label='Stay rate (ℓ=1→d=1)', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Rival Incumbents')
axes[1].set_ylabel('Rate')
axes[1].set_title('(b) Entry/Stay Rates vs Competition')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1)

# 3. Operate rate by demand shock
operate_by_s_inc = data[data['own_incumbent'] == 1].groupby('s')['choice'].mean()
operate_by_s_ent = data[data['own_incumbent'] == 0].groupby('s')['choice'].mean()

width = 0.35
x = np.arange(game.params.n_s)
axes[2].bar(x - width/2, operate_by_s_inc.values, width, label='Incumbent', color='blue', alpha=0.7)
axes[2].bar(x + width/2, operate_by_s_ent.values, width, label='Entrant', color='red', alpha=0.7)
axes[2].set_xlabel('Demand Shock State (s)')
axes[2].set_ylabel('P(Operate)')
axes[2].set_title('(c) Operate Rate by Demand')
axes[2].set_xticks(x)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print sample statistics
print("\nSimulated Data Summary:")
print("-" * 60)
print(f"Markets: {data['market_id'].nunique()}, Periods: {data['period'].nunique()}")
print(f"Total firm-period observations: {len(data):,}")
print(f"\nChoice frequencies:")
print(f"  Overall operate rate: {data['choice'].mean():.3f}")
print(f"  Incumbent stay rate:  {data[data['own_incumbent']==1]['choice'].mean():.3f}")
print(f"  Entrant enter rate:   {data[data['own_incumbent']==0]['choice'].mean():.3f}")


---

## Part 5: Estimation Methods

We implement three estimators that exploit the CCP structure:

| Method | Description | Computation |
|--------|-------------|-------------|
| **Two-Stage** | Estimate CCPs first, then structural parameters | Fast, but ignores first-stage error |
| **NPL** | Iterate between CCP and parameter updates | Moderate, consistent under regularity |
| **NFXP** | Solve equilibrium at each parameter guess | Slow, but fully efficient |

### 5.1 First Stage: CCP Estimation

We estimate CCPs non-parametrically using logistic regression:
$$P(d=1 | \text{state}) = \Lambda(\gamma' Z)$$
where $Z$ includes state variables and interactions.


In [ ]:
class FirstStage:
    """Estimates CCPs and transitions from data."""
    
    def __init__(self, data: pd.DataFrame, state_space: StateSpace):
        self.data = data
        self.state_space = state_space
    
    def estimate_ccps_frequency(self) -> np.ndarray:
        """Estimate CCPs using frequency estimator.
        
        p(x) = (# obs with choice=1 at state x) / (# obs at state x)
        """
        n_states = self.state_space.n_states
        ccps = np.full(n_states, 0.5)  # Default for states with no data
        
        for idx in range(n_states):
            x1, own_inc, n_rivals, s = self.state_space.index_to_state(idx)
            
            mask = ((self.data['x1'] == x1) & 
                    (self.data['own_incumbent'] == own_inc) &
                    (self.data['n_rivals'] == n_rivals) &
                    (self.data['s'] == s))
            
            if mask.sum() > 0:
                ccps[idx] = self.data.loc[mask, 'choice'].mean()
        
        # Clip to avoid log(0) issues
        ccps = np.clip(ccps, 0.01, 0.99)
        return ccps
    
    def estimate_ccps_logit(self) -> Tuple[np.ndarray, np.ndarray]:
        """Estimate CCPs using logit with state covariates.
        
        Flexible specification: 
        P(d=1) = logit(gamma_0 + gamma_1*x1 + gamma_2*s + gamma_3*n_rivals + 
                       gamma_4*own_inc + interactions)
        
        Returns:
            ccps: Predicted CCPs for all states
            coefs: Logit coefficients
        """
        # Build design matrix
        X = np.column_stack([
            np.ones(len(self.data)),
            self.data['x1'].values,
            self.data['s'].values,
            self.data['n_rivals'].values,
            self.data['own_incumbent'].values,
            self.data['x1'].values * self.data['own_incumbent'].values,
            self.data['s'].values * self.data['own_incumbent'].values,
            self.data['n_rivals'].values * self.data['own_incumbent'].values
        ])
        y = self.data['choice'].values
        
        # Estimate logit
        def neg_log_lik(gamma):
            linear = np.clip(X @ gamma, -700, 700)
            ll = y * linear - np.log(1 + np.exp(linear))
            return -ll.sum()
        
        def grad(gamma):
            linear = np.clip(X @ gamma, -700, 700)
            p = logit_prob(linear)
            return -X.T @ (y - p)
        
        result = minimize(neg_log_lik, np.zeros(X.shape[1]), 
                         method='BFGS', jac=grad, options={'disp': False})
        coefs = result.x
        
        # Predict CCPs for all states
        n_states = self.state_space.n_states
        ccps = np.zeros(n_states)
        
        for idx in range(n_states):
            x1, own_inc, n_rivals, s = self.state_space.index_to_state(idx)
            x_pred = np.array([1, x1, s, n_rivals, own_inc,
                              x1 * own_inc, s * own_inc, n_rivals * own_inc])
            ccps[idx] = logit_prob(x_pred @ coefs)
        
        return ccps, coefs
    
    def estimate_transition_pi(self) -> float:
        """Estimate demand shock persistence from data."""
        # Create lagged s
        data_sorted = self.data.sort_values(['market_id', 'firm_id', 'period'])
        data_sorted['s_lag'] = data_sorted.groupby(['market_id', 'firm_id'])['s'].shift(1)
        data_sorted = data_sorted.dropna()
        
        # P(s' = s | s)
        same_state = (data_sorted['s'] == data_sorted['s_lag']).mean()
        return same_state

In [ ]:
# Estimate CCPs from data
print("First Stage: Estimating CCPs from data...")
print("=" * 50)

first_stage = FirstStage(data, game.state_space)

# Frequency estimator
ccps_freq = first_stage.estimate_ccps_frequency()
print(f"\nFrequency estimator: {len(ccps_freq)} CCPs estimated")

# Logit estimator
ccps_logit, logit_coefs = first_stage.estimate_ccps_logit()
print(f"Logit estimator: {len(ccps_logit)} CCPs estimated")
print(f"  Logit coefficients: {logit_coefs.round(4)}")

# Transition estimation
pi_hat = first_stage.estimate_transition_pi()
print(f"\nEstimated pi (persistence): {pi_hat:.4f} (true: {game.params.pi})")

# Compare estimated vs true CCPs
print(f"\nCCP Comparison (sample states):")
print(f"{'State':<25} {'True':<10} {'Freq':<10} {'Logit':<10}")
print("-" * 55)
sample_states = [(2, 1, 2, 1), (2, 0, 2, 1), (3, 1, 3, 2), (0, 0, 0, 0)]
for state in sample_states:
    idx = game.state_space.state_to_index(state)
    print(f"{str(state):<25} {eq_result.ccps[idx]:<10.4f} {ccps_freq[idx]:<10.4f} {ccps_logit[idx]:<10.4f}")

### 5.2 Moment Conditions (for GMM/OLS estimation)

In [ ]:
class MomentConditions:
    """Constructs moment conditions from the identifying equation.
    
    The identifying equation (using finite dependence) is:
    
    log(p_1/(1-p_1)) = u_1(x; theta) + beta * [continuation_if_operate - continuation_if_exit]
    
    Rearranging:
    Y(x) = theta_0 + theta_1*x1 + theta_2*s + theta_3*E[n_rivals] + theta_4*is_entrant
    
    where Y(x) = log(p_1/(1-p_1)) - beta * continuation_term
    """
    
    def __init__(self, game: EntryExitGame, beta: float):
        self.game = game
        self.beta = beta
        self.state_space = game.state_space
    
    def construct_Y(self, ccps: np.ndarray) -> np.ndarray:
        """Construct LHS of identifying equation.
        
        Y(x) = log(p/(1-p)) - beta * [E[V|operate] - E[V|exit]]
        """
        # Log-odds
        log_odds_vals = log_odds(ccps)
        
        # Continuation value difference
        cv_diff = self.game.compute_continuation_value(ccps)
        
        return log_odds_vals - self.beta * cv_diff
    
    def construct_X(self, ccps: np.ndarray) -> np.ndarray:
        """Construct RHS regressors.
        
        X = [1, x1, s, E[n_active_rivals], is_entrant]
        """
        n_states = self.state_space.n_states
        X = np.zeros((n_states, 5))
        
        for idx in range(n_states):
            x1, own_inc, n_rival_inc, s = self.state_space.index_to_state(idx)
            n_rival_ent = self.game.params.I - 1 - n_rival_inc
            is_entrant = 1 - own_inc
            
            # Get average CCPs for rivals
            p_stay_avg = 0.0
            p_enter_avg = 0.0
            for n_r in range(self.game.params.I):
                idx_inc = self.state_space.state_to_index((x1, 1, n_r, s))
                idx_ent = self.state_space.state_to_index((x1, 0, n_r, s))
                p_stay_avg += ccps[idx_inc]
                p_enter_avg += ccps[idx_ent]
            p_stay_avg /= self.game.params.I
            p_enter_avg /= self.game.params.I
            
            # Expected number of active rivals
            exp_n_active = n_rival_inc * p_stay_avg + n_rival_ent * p_enter_avg
            
            X[idx] = [1, x1, s, exp_n_active, is_entrant]
        
        return X
    
    def moments(self, theta: np.ndarray, ccps: np.ndarray) -> np.ndarray:
        """Compute moment conditions: g(theta) = Y - X*theta."""
        Y = self.construct_Y(ccps)
        X = self.construct_X(ccps)
        return Y - X @ theta

### 5.3 Two-Stage Estimator

In [ ]:
import time

@dataclass
class EstimationResult:
    """Results from structural estimation.
    
    Note: Standard errors should be computed via Monte Carlo simulation,
    not analytically. The analytical SEs in two-stage estimators are invalid
    because they ignore first-stage estimation error.
    """
    theta_hat: np.ndarray
    computation_time: float = 0.0
    n_iterations: int = 0
    converged: bool = True
    method: str = ""
    additional_info: Dict = field(default_factory=dict)


class TwoStageEstimator:
    """Two-stage CCP estimator (no iteration).
    
    Stage 1: Estimate CCPs from data
    Stage 2: OLS/GMM using moment conditions
    
    Note: Standard errors should be computed via Monte Carlo, not analytically.
    """
    
    def __init__(self, game: EntryExitGame, data: pd.DataFrame):
        self.game = game
        self.data = data
        self.first_stage = FirstStage(data, game.state_space)
    
    def estimate(self, use_logit_ccps: bool = True) -> EstimationResult:
        """Run two-stage estimation.
        
        Args:
            use_logit_ccps: If True, use logit-smoothed CCPs; else frequency
        """
        start_time = time.time()
        
        # Stage 1: Estimate CCPs
        if use_logit_ccps:
            ccps_hat, _ = self.first_stage.estimate_ccps_logit()
        else:
            ccps_hat = self.first_stage.estimate_ccps_frequency()
        
        # Stage 2: OLS on moment conditions
        moments = MomentConditions(self.game, self.game.params.beta)
        Y = moments.construct_Y(ccps_hat)
        X = moments.construct_X(ccps_hat)
        
        # Weight by number of observations per state
        weights = np.zeros(self.game.state_space.n_states)
        for idx in range(self.game.state_space.n_states):
            x1, own_inc, n_rivals, s = self.game.state_space.index_to_state(idx)
            mask = ((self.data['x1'] == x1) & 
                    (self.data['own_incumbent'] == own_inc) &
                    (self.data['n_rivals'] == n_rivals) &
                    (self.data['s'] == s))
            weights[idx] = mask.sum()
        
        # WLS: theta = (X'WX)^{-1} X'WY
        W = np.diag(weights)
        XtWX = X.T @ W @ X
        XtWY = X.T @ W @ Y
        
        try:
            theta_hat = np.linalg.solve(XtWX, XtWY)
        except np.linalg.LinAlgError:
            theta_hat = np.linalg.lstsq(XtWX, XtWY, rcond=None)[0]
        
        computation_time = time.time() - start_time
        
        return EstimationResult(
            theta_hat=theta_hat,
            computation_time=computation_time,
            method='Two-Stage CCP',
            additional_info={'ccps_hat': ccps_hat}
        )

In [ ]:
# Run two-stage estimation
print("Two-Stage CCP Estimation")
print("=" * 50)

estimator_2s = TwoStageEstimator(game, data)
result_2s = estimator_2s.estimate(use_logit_ccps=True)

print(f"\nComputation time: {result_2s.computation_time:.3f} sec\n")

# Format results like Table II in the paper
param_names = ['θ₀ (intercept)', 'θ₁ (mkt char)', 'θ₂ (demand)', 
               'θ₃ (competitors)', 'θ₄ (entry cost)']
true_theta = game.params.theta_vector()

table_data = []
for i, name in enumerate(param_names):
    table_data.append([name, f"{true_theta[i]:.4f}", f"{result_2s.theta_hat[i]:.4f}"])

print(tabulate(table_data, 
               headers=['Parameter', 'DGP', 'Two-Stage'],
               tablefmt='simple',
               colalign=('left', 'right', 'right')))

print("\nNote: Standard errors should be computed via Monte Carlo (see Part 6).")


### 5.4 NPL (CCP-Model) Estimator

In [ ]:
import time

class NPLEstimator:
    """Nested Pseudo-Likelihood (NPL) / CCP-Model estimator.
    
    Iterates between:
    1. Given CCPs, estimate theta via maximum likelihood
    2. Given theta, update CCPs using model predictions
    
    Note: Standard errors should be computed via Monte Carlo simulation,
    not analytically. The analytical SEs in NPL ignore first-stage error.
    """
    
    def __init__(self, game: EntryExitGame, data: pd.DataFrame,
                 config: EstimationConfig = None):
        self.game = game
        self.data = data
        self.config = config or EstimationConfig()
        self.first_stage = FirstStage(data, game.state_space)
        
        # Precompute state indices for faster iteration
        self._precompute_data_indices()
    
    def _precompute_data_indices(self):
        """Precompute state indices and data arrays for efficient computation."""
        self.y = self.data['choice'].values
        self.x1_arr = self.data['x1'].values.astype(int)
        self.own_inc_arr = self.data['own_incumbent'].values.astype(int)
        self.n_rivals_arr = self.data['n_rivals'].values.astype(int)
        self.s_arr = self.data['s'].values.astype(int)
        
        self.state_indices = np.array([
            self.game.state_space.state_to_index(
                (self.x1_arr[i], self.own_inc_arr[i], self.n_rivals_arr[i], self.s_arr[i])
            )
            for i in range(len(self.data))
        ])
    
    def estimate(self, initial_ccps: np.ndarray = None) -> EstimationResult:
        """Run NPL estimation."""
        start_time = time.time()
        
        # Initialize CCPs
        if initial_ccps is None:
            ccps, _ = self.first_stage.estimate_ccps_logit()
        else:
            ccps = initial_ccps.copy()
        
        theta = self.game.params.theta_vector()  # Starting values
        
        ccp_history = [ccps.copy()]
        theta_history = [theta.copy()]
        
        for outer_iter in range(self.config.max_iter):
            ccps_old = ccps.copy()
            theta_old = theta.copy()
            
            # Step 1: Given CCPs, compute continuation values and estimate theta
            cv_diff = self.game.compute_continuation_value(ccps)
            
            # Construct design for each observation (vectorized)
            X_obs = np.zeros((len(self.data), 5))
            cv_obs = cv_diff[self.state_indices]
            
            for i in range(len(self.data)):
                x1 = self.x1_arr[i]
                own_inc = self.own_inc_arr[i]
                n_rivals = self.n_rivals_arr[i]
                s = self.s_arr[i]
                
                # Get expected number of active rivals
                n_rival_ent = self.game.params.I - 1 - n_rivals
                p_stay_avg = np.mean([ccps[self.game.state_space.state_to_index((x1, 1, nr, s))] 
                                      for nr in range(self.game.params.I)])
                p_enter_avg = np.mean([ccps[self.game.state_space.state_to_index((x1, 0, nr, s))] 
                                       for nr in range(self.game.params.I)])
                exp_n_active = n_rivals * p_stay_avg + n_rival_ent * p_enter_avg
                
                is_entrant = 1 - own_inc
                X_obs[i] = [1, x1, s, exp_n_active, is_entrant]
            
            # Logit MLE
            def neg_log_lik(theta_):
                v1 = X_obs @ theta_ + self.game.params.beta * cv_obs
                v1 = np.clip(v1, -700, 700)
                ll = self.y * v1 - np.log(1 + np.exp(v1))
                return -ll.sum()
            
            def grad(theta_):
                v1 = X_obs @ theta_ + self.game.params.beta * cv_obs
                v1 = np.clip(v1, -700, 700)
                p = logit_prob(v1)
                return -X_obs.T @ (self.y - p)
            
            result = minimize(neg_log_lik, theta, method='BFGS', jac=grad,
                            options={'disp': False})
            theta = result.x
            
            # Step 2: Given theta, update CCPs using model
            temp_params = ModelParameters.from_vector(
                theta, beta=self.game.params.beta, pi=self.game.params.pi,
                I=self.game.params.I, n_x1=self.game.params.n_x1, n_s=self.game.params.n_s
            )
            temp_game = EntryExitGame(temp_params)
            ccps = temp_game.ccp_operator(ccps)
            
            ccp_history.append(ccps.copy())
            theta_history.append(theta.copy())
            
            # Check convergence
            ccp_diff = np.max(np.abs(ccps - ccps_old))
            theta_diff = np.max(np.abs(theta - theta_old))
            
            if self.config.verbose and (outer_iter + 1) % 5 == 0:
                print(f"  Iter {outer_iter + 1}: theta_diff={theta_diff:.2e}, ccp_diff={ccp_diff:.2e}")
            
            if ccp_diff < self.config.tolerance and theta_diff < self.config.tolerance:
                if self.config.verbose:
                    print(f"  Converged in {outer_iter + 1} iterations")
                break
        
        computation_time = time.time() - start_time
        
        return EstimationResult(
            theta_hat=theta,
            computation_time=computation_time,
            n_iterations=outer_iter + 1,
            method='NPL (CCP-Model)',
            additional_info={'ccps_final': ccps, 'ccp_history': ccp_history}
        )

In [ ]:
# Run NPL estimation
print("NPL (CCP-Model) Estimation")
print("=" * 50)

estimator_npl = NPLEstimator(game, data, EstimationConfig(verbose=True, tolerance=1e-6))
result_npl = estimator_npl.estimate()

print(f"\nComputation time: {result_npl.computation_time:.3f} sec")
print(f"NPL iterations: {result_npl.n_iterations}\n")

# Format results like Table II
table_data = []
for i, name in enumerate(param_names):
    table_data.append([name, f"{true_theta[i]:.4f}", f"{result_npl.theta_hat[i]:.4f}"])

print(tabulate(table_data, 
               headers=['Parameter', 'DGP', 'NPL'],
               tablefmt='simple',
               colalign=('left', 'right', 'right')))

print("\nNote: Standard errors should be computed via Monte Carlo (see Part 6).")


### 5.5 NFXP Estimator

In [ ]:
import time

class NFXPEstimator:
    """Nested Fixed Point (NFXP) estimator.
    
    Outer loop: optimize over theta
    Inner loop: solve for equilibrium CCPs given theta
    
    Note: Standard errors should be computed via Monte Carlo simulation.
    """
    
    def __init__(self, game: EntryExitGame, data: pd.DataFrame,
                 solver_config: EstimationConfig = None):
        self.game = game
        self.data = data
        self.solver_config = solver_config or EstimationConfig(verbose=False)
        
        # Pre-compute data indices
        self.y = data['choice'].values
        self.state_indices = np.array([
            game.state_space.state_to_index(
                (int(row['x1']), int(row['own_incumbent']), int(row['n_rivals']), int(row['s']))
            )
            for _, row in data.iterrows()
        ])
        
        self.n_equilibrium_solves = 0
        self.n_likelihood_evals = 0
    
    def log_likelihood(self, theta: np.ndarray) -> float:
        """Compute log-likelihood for given theta.
        
        1. Create game with theta
        2. Solve for equilibrium CCPs
        3. Compute likelihood
        """
        self.n_likelihood_evals += 1
        
        # Create game with current theta
        params = ModelParameters.from_vector(
            theta, beta=self.game.params.beta, pi=self.game.params.pi,
            I=self.game.params.I, n_x1=self.game.params.n_x1, n_s=self.game.params.n_s
        )
        temp_game = EntryExitGame(params)
        
        # Solve for equilibrium
        solver = EquilibriumSolver(temp_game, self.solver_config)
        eq_result = solver.solve()
        self.n_equilibrium_solves += 1
        
        if not eq_result.converged:
            return -1e10  # Penalize non-convergence
        
        # Compute log-likelihood
        ccps = eq_result.ccps[self.state_indices]
        ccps = np.clip(ccps, 1e-10, 1 - 1e-10)
        
        ll = (self.y * np.log(ccps) + (1 - self.y) * np.log(1 - ccps)).sum()
        return ll
    
    def estimate(self, initial_theta: np.ndarray = None,
                method: str = 'Nelder-Mead') -> EstimationResult:
        """Run NFXP estimation."""
        start_time = time.time()
        self.n_equilibrium_solves = 0
        self.n_likelihood_evals = 0
        
        if initial_theta is None:
            initial_theta = self.game.params.theta_vector()
        
        # Minimize negative log-likelihood
        def neg_ll(theta):
            ll = self.log_likelihood(theta)
            return -ll
        
        result = minimize(neg_ll, initial_theta, method=method,
                         options={'disp': False, 'maxiter': 100})
        
        theta_hat = result.x
        computation_time = time.time() - start_time
        
        return EstimationResult(
            theta_hat=theta_hat,
            computation_time=computation_time,
            n_iterations=result.nit,
            converged=result.success,
            method='NFXP',
            additional_info={
                'n_equilibrium_solves': self.n_equilibrium_solves,
                'n_likelihood_evals': self.n_likelihood_evals
            }
        )

In [ ]:
# Run NFXP estimation (this is slow!)
print("NFXP Estimation")
print("=" * 50)
print("(This may take a while due to inner equilibrium solves...)\n")

# Use a smaller tolerance for inner solver to speed things up
inner_config = EstimationConfig(verbose=False, tolerance=1e-6, max_iter=100)
estimator_nfxp = NFXPEstimator(game, data, solver_config=inner_config)

# Start from true values to speed up (in practice would use 2-stage estimates)
result_nfxp = estimator_nfxp.estimate(initial_theta=true_theta)

print(f"\nComputation time: {result_nfxp.computation_time:.3f} sec")
print(f"Equilibrium solves: {result_nfxp.additional_info['n_equilibrium_solves']}")
print(f"Likelihood evaluations: {result_nfxp.additional_info['n_likelihood_evals']}\n")

# Format results like Table II
table_data = []
for i, name in enumerate(param_names):
    table_data.append([name, f"{true_theta[i]:.4f}", f"{result_nfxp.theta_hat[i]:.4f}"])

print(tabulate(table_data, 
               headers=['Parameter', 'DGP', 'NFXP'],
               tablefmt='simple',
               colalign=('left', 'right', 'right')))

print("\nNote: Standard errors should be computed via Monte Carlo (see Part 6).")


---

## Part 6: Comparison and Monte Carlo

### Why Monte Carlo?

Standard errors for CCP-based estimators are complex because:
1. First-stage estimation error propagates to structural parameters
2. Analytical formulas may not account for equilibrium constraints

**Monte Carlo simulation** provides valid standard errors by:
1. Generating many datasets from the true DGP
2. Estimating parameters on each dataset
3. Computing empirical standard deviation across replications


### Comparing Estimation Methods

The following figure compares the three estimators. **What to look for:**

- **(a) Accuracy**: All methods should recover parameters close to true values (green bars)
- **(b) Computational cost**: Two-Stage is fastest (no iteration), NFXP slowest (solves equilibrium each guess)

In this single-sample comparison, differences are small. Monte Carlo (Part 6) assesses variability.


In [ ]:
# Compare all estimation methods - Table II style
print("\nEstimation Methods Comparison (Table II Format)")
print("=" * 70)

results = {
    'Two-Stage': result_2s,
    'NPL': result_npl,
    'NFXP': result_nfxp
}

# Build table data - rows are parameters, columns are methods
# Format matching Table II: Parameter | DGP | Two-Stage | NPL | NFXP
table_data = []
for i, name in enumerate(param_names):
    row = [
        name,
        f"{true_theta[i]:.4f}",
        f"{result_2s.theta_hat[i]:.4f}",
        f"{result_npl.theta_hat[i]:.4f}",
        f"{result_nfxp.theta_hat[i]:.4f}"
    ]
    table_data.append(row)

print(tabulate(table_data,
               headers=['Parameter', 'DGP', 'Two-Stage', 'NPL', 'NFXP'],
               tablefmt='simple',
               colalign=('left', 'right', 'right', 'right', 'right')))

# Computation time summary
print("\n")
time_data = [
    ['Time (sec)', '', 
     f"{result_2s.computation_time:.2f}", 
     f"{result_npl.computation_time:.2f}", 
     f"{result_nfxp.computation_time:.2f}"],
    ['Iterations', '',
     f"{result_2s.n_iterations}",
     f"{result_npl.n_iterations}",
     f"{result_nfxp.n_iterations}"]
]
print(tabulate(time_data,
               headers=['', '', 'Two-Stage', 'NPL', 'NFXP'],
               tablefmt='simple',
               colalign=('left', 'right', 'right', 'right', 'right')))


In [ ]:
# Visualize estimation results comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Parameter estimates by method
x = np.arange(len(param_names))
width = 0.2

bars_true = axes[0].bar(x - 1.5*width, true_theta, width, label='True (DGP)', color='green', alpha=0.8)
bars_2s = axes[0].bar(x - 0.5*width, result_2s.theta_hat, width, label='Two-Stage', color='blue', alpha=0.7)
bars_npl = axes[0].bar(x + 0.5*width, result_npl.theta_hat, width, label='NPL', color='orange', alpha=0.7)
bars_nfxp = axes[0].bar(x + 1.5*width, result_nfxp.theta_hat, width, label='NFXP', color='red', alpha=0.7)

axes[0].set_ylabel('Parameter Value')
axes[0].set_title('(a) Parameter Estimates vs True Values')
axes[0].set_xticks(x)
axes[0].set_xticklabels([r'$\theta_0$', r'$\theta_1$', r'$\theta_2$', r'$\theta_3$', r'$\theta_4$'])
axes[0].legend(loc='upper right')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3, axis='y')

# 2. Computation time comparison  
methods = list(results.keys())
times = [results[m].computation_time for m in methods]
colors = ['blue', 'orange', 'red']
bars = axes[1].bar(methods, times, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Computation Time (seconds)')
axes[1].set_title('(b) Computational Cost by Method')
axes[1].grid(True, alpha=0.3, axis='y')

# Add time labels on bars
for bar, time in zip(bars, times):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 f'{time:.2f}s', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Print interpretation
print("\nFigure Interpretation:")
print("-" * 60)
print("(a) All three estimators recover true parameters accurately")
print("    - Green bars (true) closely matched by estimation bars")
print("    - Largest parameter θ₄=-1.5 (entry cost) well-identified")
print("(b) Two-Stage is fastest; NFXP slowest due to equilibrium solving")


### Monte Carlo Study (Simplified)

In [ ]:
import time

def run_monte_carlo(game: EntryExitGame, eq_ccps: np.ndarray,
                    sim_config: SimulationConfig, n_replications: int = 20,
                    methods: List[str] = ['Two-Stage', 'NPL']) -> pd.DataFrame:
    """Run Monte Carlo simulation.
    
    Args:
        game: Game with true parameters
        eq_ccps: Equilibrium CCPs
        sim_config: Simulation configuration
        n_replications: Number of Monte Carlo replications
        methods: List of methods to compare
        
    Returns:
        DataFrame with results
    """
    results = []
    true_theta = game.params.theta_vector()
    
    for rep in tqdm(range(n_replications), desc='Monte Carlo'):
        # Simulate data with different seed
        config = SimulationConfig(
            M=sim_config.M, T=sim_config.T, burn_in=sim_config.burn_in,
            seed=rep * 1000
        )
        simulator = DataSimulator(game, eq_ccps, config)
        data_rep = simulator.simulate()
        
        for method in methods:
            try:
                if method == 'Two-Stage':
                    estimator = TwoStageEstimator(game, data_rep)
                    result = estimator.estimate()
                elif method == 'NPL':
                    estimator = NPLEstimator(game, data_rep, 
                                           EstimationConfig(verbose=False, tolerance=1e-5))
                    result = estimator.estimate()
                else:
                    continue
                
                # Verify result is EstimationResult
                if not hasattr(result, 'theta_hat'):
                    print(f"Rep {rep}, {method}: result has no theta_hat, type={type(result)}")
                    continue
                
                for i, name in enumerate(param_names):
                    results.append({
                        'replication': rep,
                        'method': method,
                        'parameter': name,
                        'true': true_theta[i],
                        'estimate': result.theta_hat[i],
                        'comp_time': result.computation_time
                    })
            except Exception as e:
                import traceback
                print(f"\nRep {rep}, {method} failed: {e}")
                traceback.print_exc()
    
    return pd.DataFrame(results)

In [ ]:
# Run Monte Carlo (reduced replications for speed)
print("Monte Carlo Simulation")
print("=" * 50)

# Verify sim_config
print(f"sim_config: M={sim_config.M}, T={sim_config.T}, burn_in={sim_config.burn_in}")
print(f"Observed periods: {sim_config.T_observed}")
print("Methods: Two-Stage, NPL\n")

# Run MC with fewer replications for testing
mc_results = run_monte_carlo(game, eq_result.ccps, sim_config, 
                             n_replications=10, methods=['Two-Stage', 'NPL'])

print(f"\nCompleted {len(mc_results)} estimation results")
if len(mc_results) > 0:
    print(f"Columns: {mc_results.columns.tolist()}")

In [ ]:
# Summarize Monte Carlo results - Table II format
def summarize_mc(mc_results: pd.DataFrame) -> pd.DataFrame:
    """Compute bias, std, RMSE for Monte Carlo results."""
    if mc_results.empty:
        print("Warning: No Monte Carlo results to summarize")
        return pd.DataFrame()
    
    summary = []
    
    for method in mc_results['method'].unique():
        for param in mc_results['parameter'].unique():
            mask = (mc_results['method'] == method) & (mc_results['parameter'] == param)
            subset = mc_results[mask]
            
            if len(subset) == 0:
                continue
                
            true_val = subset['true'].iloc[0]
            estimates = subset['estimate'].values
            
            bias = (estimates - true_val).mean()
            std = estimates.std() if len(estimates) > 1 else 0.0
            rmse = np.sqrt(((estimates - true_val) ** 2).mean())
            
            summary.append({
                'Method': method,
                'Parameter': param,
                'True': true_val,
                'Mean': estimates.mean(),
                'Bias': bias,
                'Std': std,
                'RMSE': rmse
            })
    
    return pd.DataFrame(summary)

mc_summary = summarize_mc(mc_results)

if mc_summary.empty:
    print("No results to display")
else:
    # Format like Table II: Parameter | DGP | Two-Stage Mean (Std) | NPL Mean (Std)
    print("\n" + "=" * 75)
    print("Monte Carlo Results (Table II Format)")
    print("=" * 75)
    print(f"Replications: 10 | Markets: {sim_config.M} | Periods: {sim_config.T_observed}")
    print("")

    # Build table in paper format - use actual param_names from earlier
    methods = mc_summary['Method'].unique()

    # Create header row
    headers = ['Parameter', 'DGP'] + [m for m in methods]

    # Create data rows with mean and (std) format like paper
    table_data = []
    for param in param_names:  # param_names already has the full labels
        row = [param]
        
        # Get DGP (true value)
        mask = mc_summary['Parameter'] == param
        if not mask.any():
            continue
        true_val = mc_summary[mask]['True'].iloc[0]
        row.append(f"{true_val:.2f}")
        
        # Get estimate for each method
        for method in methods:
            method_mask = (mc_summary['Method'] == method) & (mc_summary['Parameter'] == param)
            if method_mask.any():
                mean = mc_summary[method_mask]['Mean'].iloc[0]
                std = mc_summary[method_mask]['Std'].iloc[0]
                # Paper format: mean with (std) below
                row.append(f"{mean:.4f}\n({std:.4f})")
            else:
                row.append("")
        
        table_data.append(row)

    print(tabulate(table_data, 
                   headers=headers,
                   tablefmt='simple',
                   colalign=['left', 'right'] + ['right'] * len(methods)))

    # Also print RMSE summary
    print("\n\nRMSE Summary:")
    rmse_data = []
    for param in param_names:
        row = [param]
        for method in methods:
            mask = (mc_summary['Method'] == method) & (mc_summary['Parameter'] == param)
            if mask.any():
                rmse = mc_summary[mask]['RMSE'].iloc[0]
                row.append(f"{rmse:.4f}")
            else:
                row.append("")
        rmse_data.append(row)

    print(tabulate(rmse_data,
                   headers=['Parameter'] + list(methods),
                   tablefmt='simple',
                   colalign=['left'] + ['right'] * len(methods)))

### Monte Carlo Distributions

The following figure shows the sampling distribution of estimates across 20 replications. **What to look for:**

- **Centering**: Distributions should be centered near the true value (green dashed line)
- **Spread**: Narrower distributions indicate more precise estimators
- **Comparison**: NPL often has slightly smaller variance than Two-Stage

These histograms justify the standard errors reported in Table II format.


In [ ]:
# Visualize Monte Carlo distributions
if mc_results.empty or len(mc_results) == 0:
    print("No Monte Carlo results to visualize")
else:
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))

    for i, param in enumerate(param_names):
        ax = axes[i // 3, i % 3]
        
        for j, method in enumerate(['Two-Stage', 'NPL']):
            mask = (mc_results['method'] == method) & (mc_results['parameter'] == param)
            estimates = mc_results.loc[mask, 'estimate'].values
            if len(estimates) > 0:
                color = 'blue' if method == 'Two-Stage' else 'orange'
                ax.hist(estimates, bins=min(12, len(estimates)), alpha=0.5, 
                        label=method, edgecolor='black', color=color)
        
        true_val = true_theta[i]
        ax.axvline(true_val, color='green', linestyle='--', linewidth=2.5, label=f'True = {true_val:.2f}')
        ax.set_xlabel('Estimate')
        ax.set_ylabel('Frequency')
        ax.legend(fontsize=8, loc='upper right')
        ax.set_title(f'{param}')

    # Hide unused subplot
    axes[1, 2].axis('off')
    axes[1, 2].text(0.5, 0.5, 'Monte Carlo validates\nconsistency:\n\n• Distributions centered\n  near true values\n\n• NPL slightly tighter\n  than Two-Stage',
                    ha='center', va='center', fontsize=11, transform=axes[1, 2].transAxes,
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.suptitle(f'Monte Carlo Distribution of Parameter Estimates ({len(mc_results) // (2*5)} replications)', y=1.02, fontsize=14)
    plt.tight_layout()
    plt.show()